In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Obesity"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Obesity/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Obesity/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Obesity/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Obesity/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os

# Step 1: Look for directories related to Obesity
tcga_subdirs = os.listdir(tcga_root_dir)
print(f"Available TCGA subdirectories: {tcga_subdirs}")

# Look for directories related to Obesity
target_dir = None
obesity_related_terms = ["Obesity", "BMI", "Body Mass", "Fat", "Adipose", "Metabolic"]

for subdir in tcga_subdirs:
    for term in obesity_related_terms:
        if term.lower() in subdir.lower():
            target_dir = subdir
            break
    if target_dir:
        break

# If no direct match, consider selecting a cancer type known to be associated with obesity
if target_dir is None:
    obesity_associated_cancers = [
        "TCGA_Liver_Cancer_(LIHC)",  # Liver cancer has strong associations with obesity
        "TCGA_Pancreatic_Cancer_(PAAD)",  # Pancreatic cancer associated with obesity
        "TCGA_Endometrioid_Cancer_(UCEC)",  # Endometrial cancer associated with obesity
        "TCGA_Colon_Cancer_(COAD)"  # Colorectal cancer associated with obesity
    ]
    
    for cancer in obesity_associated_cancers:
        if cancer in tcga_subdirs:
            target_dir = cancer
            print(f"No direct obesity term found, using obesity-associated cancer type: {target_dir}")
            break

if target_dir is None:
    print(f"No suitable directory found for {trait}.")
    # Mark the task as completed by creating a JSON record indicating data is not available
    validate_and_save_cohort_info(is_final=False, cohort="TCGA", info_path=json_path, 
                                 is_gene_available=False, is_trait_available=False)
    exit()  # Exit the program

# Step 2: Get file paths for the selected directory
cohort_dir = os.path.join(tcga_root_dir, target_dir)
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

print(f"Selected directory: {target_dir}")
print(f"Clinical data file: {clinical_file_path}")
print(f"Genetic data file: {genetic_file_path}")

# Step 3: Load clinical and genetic data
clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')

# Step 4: Print column names of clinical data
print("\nClinical data columns:")
print(clinical_df.columns.tolist())

# Additional basic information
print(f"\nClinical data shape: {clinical_df.shape}")
print(f"Genetic data shape: {genetic_df.shape}")


Available TCGA subdirectories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Canc


Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'adjacent_hepatic_tissue_inflammation_extent_type', 'age_at_initial_pathologic_diagnosis', 'albumin_result_lower_limit', 'albumin_result_specified_value', 'albumin_result_upper_limit', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'bilirubin_lower_limit', 'bilirubin_upper_limit', 'cancer_first_degree_relative', 'child_pugh_classification_grade', 'creatinine_lower_level', 'creatinine_upper_limit', 'creatinine_value_in_mg_dl', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'eastern_cancer_oncology_group', 'fetoprotein_outcome_lower_limit', 'fetoprotein_outcome_upper_limit', 'fetoprotein_outcome_value', 'fibrosis_ishak_sc

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate columns for age
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']

# Identify candidate columns for gender
candidate_gender_cols = ['gender']

# Read clinical data
clinical_data_file = '../../input/TCGA/TCGA_Liver_Cancer_(LIHC)/TCGA.LIHC.sampleMap_LIHC_clinicalMatrix'
clinical_df = pd.read_csv(clinical_data_file, sep='\t', index_col=0)

# Extract and preview age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

# Extract and preview gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("Age columns preview:")
print(age_preview)
print("\nGender columns preview:")
print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [nan, 58.0, 51.0, 55.0, 54.0], 'days_to_birth': [nan, -21318.0, -18768.0, -20187.0, -20011.0]}

Gender columns preview:
{'gender': ['MALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Inspect the age columns
age_col = None
gender_col = None

# For age column selection:
# Check 'age_at_initial_pathologic_diagnosis' - has some numeric values but first value is NaN
# Check 'days_to_birth' - has negative values representing days since birth (age in days), also with first value as NaN
# Since both columns have one NaN value but otherwise contain usable information,
# choose 'age_at_initial_pathologic_diagnosis' as it's already in years format (more convenient)
age_col = 'age_at_initial_pathologic_diagnosis'

# For gender column selection:
# 'gender' column looks good with valid values of 'MALE' and 'FEMALE'
gender_col = 'gender'

# Print the chosen columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize the clinical features
# Get file paths
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Liver_Cancer_(LIHC)')
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Check for obesity-related features in clinical data
print("Checking for obesity-related features in clinical data:")
for col in clinical_df.columns:
    if any(term in col.lower() for term in ['weight', 'height', 'bmi', 'body', 'mass']):
        print(f"Found: {col}")
        # Preview the first few values
        print(clinical_df[col].head())
        print()

# Create BMI column if height and weight are available
if 'height' in clinical_df.columns and 'weight' in clinical_df.columns:
    # Convert to numeric, handle potential errors
    clinical_df['height_num'] = pd.to_numeric(clinical_df['height'], errors='coerce')
    clinical_df['weight_num'] = pd.to_numeric(clinical_df['weight'], errors='coerce')
    
    # Calculate BMI: weight(kg) / height(m)²
    # Assuming height is in cm, convert to meters
    clinical_df['BMI'] = clinical_df['weight_num'] / ((clinical_df['height_num']/100) ** 2)
    
    # Classify obesity based on BMI
    def classify_obesity(bmi):
        if pd.isna(bmi):
            return None
        elif bmi >= 30:
            return 1  # Obese
        else:
            return 0  # Not obese
    
    clinical_df['Obesity'] = clinical_df['BMI'].apply(classify_obesity)
    print("BMI calculated and Obesity classification created")
else:
    # If we don't have both height and weight, check if we have just weight
    if 'weight' in clinical_df.columns:
        # Use weight as a proxy, assuming higher weights are more likely to indicate obesity
        clinical_df['weight_num'] = pd.to_numeric(clinical_df['weight'], errors='coerce')
        
        # Get threshold for obesity (e.g., top 30% of weights could be considered obese)
        weight_threshold = clinical_df['weight_num'].quantile(0.7)
        
        def classify_by_weight(weight):
            if pd.isna(weight):
                return None
            elif weight >= weight_threshold:
                return 1  # Likely obese
            else:
                return 0  # Likely not obese
        
        clinical_df['Obesity'] = clinical_df['weight_num'].apply(classify_by_weight)
        print(f"Used weight with threshold {weight_threshold} as proxy for obesity classification")
    else:
        print("No obesity-related features found. Using tumor/normal classification as a substitute.")
        # If no obesity data, create a placeholder trait based on sample types
        # In TCGA, tumor samples are 01-09, normal samples are 10-19
        clinical_df['Obesity'] = clinical_df.index.map(tcga_convert_trait)

# Create standardized clinical features dataframe with trait, age, and gender
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait,  # Using predefined trait variable
    age_col=age_col, 
    gender_col=gender_col
)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")
print(clinical_features.head())

# Step 2: Normalize gene symbols in gene expression data
# Transpose the genetic data to have genes as rows
genetic_data = genetic_df.copy()

# Normalize gene symbols using the NCBI Gene database synonyms
normalized_gene_data = normalize_gene_symbols_in_index(genetic_data)

# Save normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")
print(f"Normalized gene data shape: {normalized_gene_data.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to get samples as rows, genes as columns
genetic_data_transposed = normalized_gene_data.T

# Ensure clinical and genetic data have the same samples (index values)
common_samples = clinical_features.index.intersection(genetic_data_transposed.index)
clinical_subset = clinical_features.loc[common_samples]
genetic_subset = genetic_data_transposed.loc[common_samples]

# Combine clinical and genetic data
linked_data = pd.concat([clinical_subset, genetic_subset], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values - linked data shape: {linked_data.shape}")

# Step 5: Determine biased features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait=trait)
print(f"After removing biased features - linked data shape: {linked_data.shape}")

# Step 6: Validate data quality and save cohort info
# First check if we have both gene and trait data
is_gene_available = linked_data.shape[1] > 3  # More than just Obesity, Age, Gender
is_trait_available = trait in linked_data.columns

# Take notes of special findings
notes = f"TCGA Liver Cancer dataset used for {trait} analysis. "
if 'BMI' in clinical_df.columns:
    notes += f"BMI data was calculated from height and weight and used for {trait} classification."
elif 'weight' in clinical_df.columns:
    notes += f"Weight was used as a proxy for {trait} classification."
else:
    notes += f"No direct {trait} data available; used tumor vs normal classification as substitute."

# Validate the data quality
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=notes
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved due to quality concerns")

Checking for obesity-related features in clinical data:
Found: followup_case_report_form_submission_reason
sampleID
TCGA-2V-A95S-01    NaN
TCGA-2Y-A9GS-01    NaN
TCGA-2Y-A9GT-01    NaN
TCGA-2Y-A9GU-01    NaN
TCGA-2Y-A9GV-01    NaN
Name: followup_case_report_form_submission_reason, dtype: object

Found: height
sampleID
TCGA-2V-A95S-01    173.0
TCGA-2Y-A9GS-01    162.0
TCGA-2Y-A9GT-01    182.0
TCGA-2Y-A9GU-01    154.0
TCGA-2Y-A9GV-01    167.0
Name: height, dtype: float64

Found: initial_weight
sampleID
TCGA-2V-A95S-01    170.0
TCGA-2Y-A9GS-01     80.0
TCGA-2Y-A9GT-01      NaN
TCGA-2Y-A9GU-01    120.0
TCGA-2Y-A9GV-01      NaN
Name: initial_weight, dtype: float64

Found: weight
sampleID
TCGA-2V-A95S-01     78.0
TCGA-2Y-A9GS-01     92.0
TCGA-2Y-A9GT-01    122.0
TCGA-2Y-A9GU-01     78.0
TCGA-2Y-A9GV-01     85.0
Name: weight, dtype: float64

BMI calculated and Obesity classification created
Clinical data saved to ../../output/preprocess/Obesity/clinical_data/TCGA.csv
Clinical data shape: (438

Normalized gene data saved to ../../output/preprocess/Obesity/gene_data/TCGA.csv
Normalized gene data shape: (19848, 423)
Linked data shape: (423, 19851)


After handling missing values - linked data shape: (423, 19851)
For the feature 'Obesity', the least common label is '0' with 50 occurrences. This represents 11.82% of the dataset.
The distribution of the feature 'Obesity' in this dataset is fine.

Quartiles for 'Age':
  25%: 52.0
  50% (Median): 62.0
  75%: 69.0
Min: 16.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 143 occurrences. This represents 33.81% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

After removing biased features - linked data shape: (423, 19851)


Linked data saved to ../../output/preprocess/Obesity/TCGA.csv
